# Example: Regression with `QNNRegressor`



## Imports

In [6]:
import numpy as np
import matplotlib.pyplot as plt
from skopt.space import Real

from squlearn import Executor
from squlearn.observables import IsingHamiltonian
from squlearn.qnn import QNNRegressor, SquaredLoss
from squlearn.optimizers import SLSQP, Adam
from src.squlearn.encoding_circuit import ChebyshevRx
from src.squlearn.optimizers import SGLBO

ImportError: cannot import name 'AuthClient' from partially initialized module 'qiskit_ibm_provider.api.clients' (most likely due to a circular import) (C:\Users\Dennis\AppData\Local\Programs\Python\Python310\lib\site-packages\qiskit_ibm_provider\api\clients\__init__.py)

We will use Qiskit's `StatevectorSimulator` for trainning.

In [ ]:
executor = Executor("statevector_simulator")

### BO Settings

In [ ]:
x0 = [[i * 0.02] for i in range(15)]
options = {"bo_aqc_func":"LCB", "bo_aqc_optimizer":"lbfgs", "bo_bounds":[Real(0.0, 0.3)], "log_file":"log_1", "bo_n_calls": 30, "bo_x0_points": x0, "bo_initial_point_generator": "random", "maxiter": 200, "bo_kappa": 0.1}

## `QNNRegressor` Setup

We start by defining a parameterized quantum circuit (PQC)

In [ ]:


nqubits = 8
number_of_layers = 4

pqc = ChebyshevRx(nqubits, 1, num_layers=number_of_layers)

We also need an observable

In [ ]:
ising_op = IsingHamiltonian(nqubits, I="S", Z="S", ZZ="S")
print(ising_op)

Now we can create the `QNNRegressor` with the PQC and cost operator. We also specify `SquaredLoss` and `SLSQP` as the loss function and optimizer respectively. Finally we specify random initial parameters for both, the PQC and cost operator.

In [ ]:
np.random.seed(13)
param_ini = np.random.rand(pqc.num_parameters)
param_op_ini = np.random.rand(ising_op.num_parameters)

reg = QNNRegressor(pqc, ising_op, executor, SquaredLoss(), SGLBO(options), param_ini, param_op_ini)

## Training

Now we are ready to train the model to fit the logarithm funciton

In [ ]:
x_space = np.arange(0.1, 0.9, 0.1)
ref_values = np.log(x_space)

reg.fit(x_space, ref_values)

## Evaluation

Let's check the trained parameters of the PQC and operator

In [ ]:
print("Result PQC params:", reg.param)
print("Result operator params:", reg.param_op)

Also let's plot the predicted function vs. the actual logarithm function

In [ ]:
x = np.arange(np.min(x_space), np.max(x_space), 0.005)
y = reg.predict(x)
plt.plot(x, np.log(x))
plt.plot(x, y)

Let's also plot the error of the QNN.

In [ ]:
plt.plot(x, np.abs(y - np.log(x)))